In [ ]:
import pandas as pd
import numpy as np
from random import shuffle
from osgeo import ogr, osr
from sentinelhub import WmsRequest, WcsRequest, MimeType, CRS, BBox, constants
from s2cloudless import S2PixelCloudDetector, CloudMaskRequest
import logging
from collections import Counter
import datetime
import os
import yaml
from sentinelhub import DataSource
import scipy.sparse as sparse
from scipy.sparse.linalg import splu


from sentinelhub import CustomUrlParam

In [ ]:
%run ../src/slope.py
%run ../src/utils.py

In [ ]:
EPSG = CRS.WGS84
GRID_SIZE_X = 1
GRID_SIZE_Y = 1

IMAGE_X = 14*GRID_SIZE_X
IMAGE_Y = 14*GRID_SIZE_Y

TEST_X = 5
TEST_Y = 5

with open("../config.yaml", 'r') as stream:
    key = (yaml.safe_load(stream))
    API_KEY = key['key']

In [ ]:
year = 2018
#time = (str(year - 1) +'-12-15', str(year+1) +'-1-15')
time = ('2018-12-15', '2019-11-25')
SIZE = 9
print(time)

def convertCoords(xy, src='', targ=''):

    srcproj = osr.SpatialReference()
    srcproj.ImportFromEPSG(src)
    targproj = osr.SpatialReference()
    if isinstance(targ, str):
        targproj.ImportFromProj4(targ)
    else:
        targproj.ImportFromEPSG(targ)
    transform = osr.CoordinateTransformation(srcproj, targproj)

    pt = ogr.Geometry(ogr.wkbPoint)
    pt.AddPoint(xy[0], xy[1])
    pt.Transform(transform)
    return([pt.GetX(), pt.GetY()])

def bounding_box(point, x_offset_max = 140, y_offset_max = 140, expansion = 10):
    # LONG, LAT FOR SOME REASON
    tl = point
    
    if 48 <= tl[0] <= 54:
        epsg = 32639 if tl[1] > 0 else 32739
    if 42 <= tl[0] <= 48:
        epsg = 32638 if tl[1] > 0 else 32738
    if 36 <= tl[0] <= 42:
        epsg = 32637 if tl[1] > 0 else 32737
    if 30 <= tl[0] <= 36:
        epsg = 32636 if tl[1] > 0 else 32736
    if 24 <= tl[0] <= 30:
        epsg = 32635 if tl[1] > 0 else 32735
    if 18 <= tl[0] <= 24:
        epsg = 32634 if tl[1] > 0 else 32734

    tl = convertCoords(tl, 4326, epsg)
    
    br = (tl[0], tl[1])
    tl = ((tl[0] + (x_offset_max)), (tl[1] + (y_offset_max )))
    distance1 = tl[0] - br[0]
    distance2 = tl[1] - br[1]
    #EXPANSION = (expansion - np.mean([distance1, distance2]))/2 # should this be 155 or 160?
    
    br = [a - expansion for a in br]
    tl = [a + expansion for a in tl]
    
    after = [b - a for a,b in zip(br, tl)]
    #br = (br[0] + 20, br[1] + 20)
    #tl = (tl[0] - 20, tl[1] - 20)
    br = convertCoords(br, epsg, 4326)
    tl = convertCoords(tl, epsg, 4326)
    
    min_x = tl[0] # original X offset - 10 meters
    max_x = br[0] # original X offset + 10*GRID_SIZE meters
    
    min_y = tl[1] # original Y offset - 10 meters
    max_y = br[1] # original Y offset + 10 meters + 140 meters
    # (min_x, min_y), (max_x, max_y)
    # (bl, tr)
    return [(min_x, min_y), (max_x, max_y)]
 
cloud_detector = S2PixelCloudDetector(threshold=0.4, average_over=4, dilation_size=2)

def identify_clouds(bbox, epsg = EPSG, time = time):
    try:
        box = BBox(bbox, crs = epsg)
        cloud_request = WmsRequest(
            layer='CLOUD_DETECTION',
            bbox=box,
            time=time,
            width=(SIZE*14)+2,
            height=(SIZE*14)+2,
            image_format = MimeType.TIFF_d32f,
            maxcc=0.5,
            instance_id=API_KEY,
            custom_url_params = {constants.CustomUrlParam.UPSAMPLING: 'NEAREST'},
            time_difference=datetime.timedelta(hours=24),
        )
        
        cloud_img = cloud_request.get_data()
        cloud_probs = cloud_detector.get_cloud_probability_maps(np.array(cloud_img))
        means = np.mean(cloud_probs, (1, 2))
        cloud_steps = [i for i, val in enumerate(means) if val > 0.20]
        return cloud_steps, means, cloud_probs
    except Exception as e:
        logging.fatal(e, exc_info=True)
    
    
def download_dem(bbox, epsg = EPSG):
    #bbox = modify_bbox(bbox, expansion = 10)
    box = BBox(bbox, crs = epsg)
    dem_s = (SIZE*14)+4
    dem_request = WmsRequest(data_source=DataSource.DEM,
                         layer='DEM',
                         bbox=box,
                         width=dem_s,
                         height=dem_s,
                         instance_id=API_KEY,
                         image_format=MimeType.TIFF_d32f,
                         custom_url_params={CustomUrlParam.SHOWLOGO: False})
    dem_image = dem_request.get_data()[0]
    dem_image = calcSlope(dem_image.reshape((1, dem_s, dem_s)),
                  np.full((dem_s, dem_s), 10), np.full((dem_s, dem_s), 10), zScale = 1, minSlope = 0.02)
    dem_image = dem_image.reshape((dem_s,dem_s, 1))
    dem_image = dem_image[1:dem_s-1, 1:dem_s-1, :]
    return dem_image #/ np.max(dem_image)


    
def download_tiles(bbox, clean_steps, epsg = EPSG, time = time):
    try:
        clean_steps = np.argwhere(clean_steps <= 0.2)
        box = BBox(bbox, crs = epsg)
        image_request = WmsRequest(
                layer='L2A',
                bbox=box,
                time=time,
                width=(SIZE*14)+2,
                height=(SIZE*14)+2,
                image_format = MimeType.TIFF_d32f,
                maxcc=0.50,
                instance_id=API_KEY,
                custom_url_params = {constants.CustomUrlParam.UPSAMPLING: 'NEAREST'},
                time_difference=datetime.timedelta(hours=24),
            )
        img_bands = image_request.get_data()
        img_bands = np.array(img_bands)
        print("There are {}/{} clean steps".format(len(clean_steps), len(img_bands)))
        return img_bands, image_request

    except Exception as e:
        logging.fatal(e, exc_info=True)
        
    
def calculate_and_save_best_images(img_bands, image_dates):

    biweekly_dates = [day for day in range(0, 360, 5)] # ideal imagery dates are every 15 days
    
    # Identify the dates where there is < 20% cloud cover
    #satisfactory_ids = list(np.argwhere(np.array(means) < 4.).reshape(-1, )) 
    satisfactory_ids = [x for x in range(0, img_bands.shape[0])]
    satisfactory_dates = [value for idx, value in enumerate(image_dates) if idx in satisfactory_ids]
    
    
    selected_images = {}
    for i in biweekly_dates:
        distances = [abs(date - i) for date in satisfactory_dates]
        closest = np.min(distances)
        closest_id = np.argmin(distances)
        # If there is imagery within 8 days, select it
        if closest < 5:
            date = satisfactory_dates[closest_id]
            image_idx = int(np.argwhere(np.array(image_dates) == date)[0])
            selected_images[i] = {'image_date': [date], 'image_ratio': [1], 'image_idx': [image_idx]}
        # If there is not imagery within 8 days, look for the closest above and below imagery
        else:
            distances = np.array([(date - i) for date in satisfactory_dates])
            # Number of days above and below the selected date of the nearest clean imagery
            above = distances[np.where(distances < 0, distances, -np.inf).argmax()]
            below = distances[np.where(distances > 0, distances, np.inf).argmin()]
            if abs(above) > 100: # If date is the last date, occassionally argmax would set above to - number
                above = below
            if abs(below) > 100:
                below = above
            if above != below:
                below_ratio = above / (above - below)
                above_ratio = 1 - below_ratio
            else:
                above_ratio = below_ratio = 0.5
                
            # Extract the image date and imagery index for the above and below values
            above_date = i + above
            above_image_idx = int(np.argwhere(np.array(image_dates) == above_date)[0])
            
            below_date = i + below
            below_image_idx = int(np.argwhere(np.array(image_dates) == below_date)[0])
            
            selected_images[i] = {'image_date': [above_date, below_date], 'image_ratio': [above_ratio, below_ratio],
                                 'image_idx': [above_image_idx, below_image_idx]}
                            
    #for i in selected_images.keys():
    #    print(i, selected_images[i])
        
    keep_steps = []
    for i in selected_images.keys():
        info = selected_images[i]
        if len(info['image_idx']) == 1:
            step = img_bands[info['image_idx'][0]]
        if len(info['image_idx']) == 2:
            step1 = img_bands[info['image_idx'][0]] * info['image_ratio'][0]
            step2 = img_bands[info['image_idx'][1]] * info['image_ratio'][1]
            step = step1 + step2
        keep_steps.append(step)
        
    keep_steps = np.stack(keep_steps)
    return keep_steps

def calc_best(tiles, cloud_probs, request, offset_x, offset_y):
    c_probs = cloud_probs[:, offset_x:offset_x+16, offset_y:offset_y+16]
    images = np.stack(tiles)[:, offset_x:offset_x+16, offset_y:offset_y+16]
    mins = np.min(c_probs, axis = 0)
    c_probs = c_probs - mins
    # Calculating the number of pixels in the 16x16 window that are 0.3 above min cloud probability
    args = np.array([len(np.argwhere(c_probs[x, :, :].reshape(16*16) > 0.3)) for x in range(c_probs.shape[0])])
    print(len([x for x in args if x > 3]))
    best = calculate_and_save_best_images(images, request, args)
    return best

def speyediff(N, d, format = 'csc'):
    shape = (N-d, N)
    diagonals = np.zeros(2*d + 1)
    diagonals[d] = 1.
    for i in range(d):
        diff = diagonals[:-1] - diagonals[1:]
        diagonals = diff
    offsets = np.arange(d+1)
    spmat = sparse.diags(diagonals, offsets, shape, format = format)
    return spmat

def smooth(y, lmbd, d = 2):
    m = len(y)
    E = sparse.eye(m, format = 'csc')
    D = speyediff(m, d, format = 'csc')
    coefmat = E + lmbd * D.conj().T.dot(D)
    z = splu(coefmat).solve(y)
    return z

In [ ]:
coords = (13.540810, 38.177220) # tigray
#coords = (-1.726374, 37.440204) # makueni
#coords = (-1.515869, 29.952997) # rwanda
coords = (coords[1], coords[0])
OUTPUT_FOLDER = '../data/tigray-l2a/'
#13.567962754335872

borders = bounding_box(coords, 10*(SIZE*14), 10*(SIZE*14), expansion = 0)
print(borders)

In [ ]:
# this needs the first coordinate to not be [coords] for different y steps

def calculate_offset_coords(coords, x_step, y_step, number):
    offset_coords = []
    y_coord = bounding_box(coords, x_step, y_step, expansion = 0)
    y_coord = y_coord[0][1]
    for i in range(number):
        bbx = bounding_box(coords, (i+1)*x_step, y_step, expansion = 10)
        coord_x = bbx[0][0]
        coord_y = y_coord
        offset_coords.append((coord_x, coord_y))
    coords = [(coords[0], y_coord)]
    return coords + offset_coords

corner_coordinates = []
for row in range(0, 30):
    temp = calculate_offset_coords(coords, x_step = (SIZE*140), 
                                            y_step = row*(SIZE*140), 
                                            number = 30)
    corner_coordinates.append([[x, [y, row]] for x, y in zip(temp, [col for col in range(0, 30)])])


In [ ]:
def check_outliers(sample):
    for date in range(24):
        for band in range(10):
            maxs = np.max(sample[date, :, :, band])
            mins = np.min(sample[date, :, :, band])
            if maxs == 1.0 or mins == 0.0:
                print("Found null outlier")
                return True
            if maxs == mins:
                print("Found missing outlier")
                return True
            if maxs >= 1.05 or mins <= -1.05:
                print("Found range outlier")
                return True
    else:
        return False
    
    
def tile_images(arr, output_folder):
    # Normal
    for x_offset, cval in enumerate([x for x in range(0, 70, 14)]):
        for y_offset, rval in enumerate([x for x in range(0, 70, 14)]):
            base_id = 0
            subs = arr[:, cval:cval+16, rval:rval+16]
            np.save(output_folder + str(base_id+x_offset*5+y_offset+1), subs)
            
    # Upright        
    for x_offset, cval in enumerate([x for x in range(7, 70, 14)]):
        for y_offset, rval in enumerate([x for x in range(7, 70, 14)]):
            base_id = 25
            subs = arr[:, cval:cval+16, rval:rval+16]
            np.save(output_folder + str(base_id+x_offset*4+y_offset+1), subs)
            
    # Right
    for x_offset, cval in enumerate([x for x in range(7, 63, 14)]):
        for y_offset, rval in enumerate([x for x in range(0, 70, 14)]):
            base_id = 25+16
            subs = arr[:, cval:cval+16, rval:rval+16]
            np.save(output_folder + str(base_id+x_offset*5+y_offset+1), subs)
            
    # Up
    for x_offset, cval in enumerate([x for x in range(0, 70, 14)]):
        for y_offset, rval in enumerate([x for x in range(7, 63, 14)]):
            base_id = 25+16+20
            subs = arr[:, cval:cval+16, rval:rval+16]
            np.save(output_folder + str(base_id+x_offset*4+y_offset+1), subs)
            
            
def calculate_proximal_steps(date, satisfactory):
    arg_before = None
    arg_after = None
    if date > 0:
        idx_before = satisfactory - date
        arg_before = idx_before[np.where(idx_before < 0, idx_before, -np.inf).argmax()]
    if date < np.max(satisfactory):
        idx_after = satisfactory - date
        arg_after = idx_after[np.where(idx_after > 0, idx_after, np.inf).argmin()]
    if not arg_after and not arg_before:
        arg_after = date
        arg_before = date
    if not arg_after:
        arg_after = arg_before
    if not arg_before:
        arg_before = arg_after
    #print(arg_before, date, arg_after)
    return arg_before, arg_after

def worker_download(coord, folder = OUTPUT_FOLDER, year = 2019):
    idx_x = coord[1][0]
    idx_y = coord[1][1]
    print("Starting: {} {}".format(idx_x, idx_y))
    coord = coord[0]
    output_folder = OUTPUT_FOLDER + "{}/{}/".format(str(idx_y), str(idx_x))
    if not os.path.exists(os.path.realpath(output_folder)):
        os.makedirs(os.path.realpath(output_folder))
    existing = len([x for x in os.listdir(os.path.realpath(output_folder))])
    if existing > 80:
        tiled_bbx = bounding_box(coord, y_offset_max = SIZE*140, x_offset_max = SIZE*140, expansion = 10)
        dem_bbx = bounding_box(coord, y_offset_max = SIZE*140, x_offset_max = SIZE*140, expansion = 20)
        cloud_steps, means, cloud_probs = identify_clouds(tiled_bbx)
        tiles, request = download_tiles(tiled_bbx, means)
        
        image_dates = []
        for date in request.get_dates():
            if date.year == year - 1:
                image_dates.append(-360 + (date.month-1)*30 + date.day)
            if date.year == year:
                image_dates.append((date.month-1)*30 + date.day)
            if date.year == year + 1:
                image_dates.append(365 + (date.month-1)*30+date.day)
        image_dates = np.array(image_dates)

        args = np.array([len(np.argwhere(cloud_probs[x, :, :].reshape((SIZE*14+2)*(SIZE*14+2)) > 0.3)) for x in range(cloud_probs.shape[0])])
        dirty_steps = np.argwhere(args > (SIZE*14+2)*(SIZE*14+2) / 10)
        #print(dirty_steps)
        missing_images = [np.argwhere(tiles[x, :, : :].flatten() == 0.0) for x in range(tiles.shape[0])]
        missing_images = [len(x) for x in missing_images]
        tiles = np.delete(tiles, dirty_steps, 0)
        cloud_probs = np.delete(cloud_probs, dirty_steps, 0)
        image_dates = np.delete(image_dates, dirty_steps)

        dem = download_dem(dem_bbx)
        dem = np.tile(dem.reshape((1, (SIZE*14)+2, (SIZE*14)+2, 1)), (tiles.shape[0], 1, 1, 1))
        tiles = np.concatenate([tiles, dem], axis = -1)

        for x_offset, cval in enumerate([x for x in range(0, SIZE*14, 14)]):
            for y_offset, rval in enumerate([x for x in range(0, SIZE*14, 14)]):
                subs = tiles[:, cval:cval+16, rval:rval+16, : ]
                c_probs = cloud_probs[:, cval:cval+16, rval:rval+16]
                args = np.array([len(np.argwhere(c_probs[x, :, :].reshape(16*16) > 0.3)) for x in range(c_probs.shape[0])])
                to_fix = list(np.argwhere(args > 10))
                to_fix = np.unique(np.array(to_fix))

                satisfactory_steps = np.array([x for x in range(tiles.shape[0]) if x not in to_fix])
                if len(to_fix) > 0:
                    for i in to_fix:
                        before, after = calculate_proximal_steps(i, satisfactory_steps)
                        bef = tiles[before, cval:cval+16, rval:rval+16, : ]
                        aft = tiles[after, cval:cval+16, rval:rval+16, : ]
                        tiles[i, cval:cval+16, rval:rval+16, : ] = (bef + aft) / 2
        
        
        tiles[:, :, :, 10] /= 90
        x, amin = evi(tiles, True)
        # Where evi is OOB, remove (likely cloud cover missed)
        if len(amin) > 0:
            satisfactory = [x for x in range(x.shape[0]) if x not in amin]
            for i in amin:
                before, after = calculate_proximal_steps(i, satisfactory)
                print("Interpolating {} with {} and {}".format(i, before, after))
                bef = x[before, :, :, :]
                aft = x[after, :, :, :]
                x[i, :, :, :] = (bef + aft) / 2
        x = bi(x, True)
        x = msavi2(x, True)
        x = si(x, True)
        
        # check for and remove null values
        missing = np.argwhere(np.isnan(np.mean(x, axis = (1, 2, 3))))
        if len(missing) > 0:
            x = np.delete(x, missing, 0)
            image_dates = np.delete(image_dates, missing)
        
        x = calculate_and_save_best_images(x, image_dates)
    
        for row in range(0, 72):
            for column in range(0, 72):
                for band in [x for x in range(0, 15) if x != 10]:
                    sm = smooth(x[:, row, column, band], 1.0, d = 2)
                    x[:, row, column, band] = sm

        biweekly_dates = np.array([day for day in range(0, 360, 5)])
        to_remove = np.argwhere(biweekly_dates % 15 != 0)
        x = np.delete(x, to_remove, 0)
        #tile_images(x, output_folder)
        return x

In [ ]:
for r in range(0, 1):
    for c in range(0, 1):
        x = worker_download(corner_coordinates[r][c])

# 0 24 has an invalid value
# we are at 1, 25

In [ ]:
d10 = np.stack([x[:, :, :, 0], x[:, :, :, 1], x[:, :, :, 2], x[:, :, :, 6]])
d20 = np.stack([x[:, :, :, 3], x[:, :, :, 4], x[:, :, :, 5], x[:, :, :, 7], 
                x[:, :, :, 8], x[:, :, :, 9]])

#3, 4, 5, 7, 8, 9

d10 = d10.reshape(24, 4, 128, 128)
d20 = d20.reshape(24, 6, 128, 128)

In [ ]:
d10.shape

In [ ]:
d20.shape

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.figure(figsize=(12, 10))
#sns.heatmap(x[0, :, :, 3] * ( (pan[0, :, :] - np.mean(x[0, :, :, 3]))))
sns.heatmap(d20[0, 1, :, :])

In [ ]:
%run ../src/dsen2/utils/DSen2Net.py

In [ ]:
# We need utils.Dsen2Net, s2model

MDL_PATH = "../src/dsen2/models/"
def DSen2(d10, d20):
    d10 = np.reshape(d10, (24, 4, 128, 128))
    d20 = np.reshape(d20, (24, 6, 128, 128))
    print(np.max(d10))
    test = [d10, d20]
    input_shape = ((4, None, None), (6, None, None))
    prediction = _predict(test, input_shape, deep=False)
    #prediction *= 5
    return prediction

def _predict(test, input_shape, deep=False, run_60=False):
    model = s2model(input_shape, num_layers=6, feature_size=128)
    predict_file = MDL_PATH+'s2_032_lr_1e-04.hdf5'
    print('Symbolic Model Created.')

    model.load_weights(predict_file)
    print("Predicting using file: {}".format(predict_file))
    prediction = model.predict(test, verbose=1)
    return prediction

pred = DSen2(d10, d20)

In [ ]:
plt.figure(figsize=(12, 10))


sns.heatmap(pred[0, 1])